In [1]:
import polars as pl
from dotenv import load_dotenv
from libraries.client_stashapp import StashAppClient, get_stashapp_client

load_dotenv()

stash_client = StashAppClient()
stash_raw_client = get_stashapp_client()

all_stash_studios = stash_client.get_studios()
all_stashapp_performers = stash_client.get_performers()

use_studio_code_tag = stash_raw_client.find_tag("Filenames: Use Studio Code")

# Lookup functions
def get_by_parent_studio(parent_studio_id):
    return all_stash_studios.filter(pl.col("stash_studios_parent_studio_id") == parent_studio_id)

def get_studio(studio_id):
    return all_stash_studios.filter(pl.col("stash_studios_id") == studio_id)

def get_studio_by_name(studio_name):
    return all_stash_studios.filter(pl.col("stash_studios_name") == studio_name)

def get_performer(performer_id):
    return all_stashapp_performers.filter(pl.col("stashapp_id") == performer_id)

dUsing stash (v0.27.2-37-g0621d871) endpoint at http://localhost:6969/graphql
dPersisting Connection to Stash with ApiKey...
dUsing stash (v0.27.2-37-g0621d871) endpoint at http://localhost:6969/graphql
dPersisting Connection to Stash with ApiKey...


In [2]:
# current_studios = get_by_parent_studio("999")
current_studios = get_studio_by_name("Blacked")
current_studio_ids = current_studios.select(pl.col("stash_studios_id")).to_series().to_list()

current_studios

stash_studios_id,stash_studios_name,stash_studios_url,stash_studios_stashdb_id,stash_studios_tpdb_id,stash_studios_ce_id,stash_studios_tags,stash_studios_parent_studio_id,stash_studios_parent_studio_name,stash_studios_parent_studio_url,stash_studios_parent_studio_tags,stash_studios_parent_studio_stashdb_id,stash_studios_parent_studio_tpdb_id,stash_studios_parent_studio_ce_id
i64,str,str,str,str,str,list[struct[2]],i64,str,str,list[struct[2]],str,str,str
18,"""Blacked""","""https://www.blacked.com/""","""324ea274-1afb-4f80-aa66-c6ddb5…","""735b22d9-aa2b-4905-8499-881a80…","""018c0783-94c9-75d4-a206-fe48eb…",[],212,"""Vixen Media Group""","""https://vixengroup.com""",[],"""b62bc449-c3d9-49ff-9a16-8f5b1b…",null,null


# Rename scenes

In [8]:
# Scan the videos
videos_paths = ["W:\\Culture\\Videos", "X:\\Culture\\Videos", "Y:\\Culture\\Videos", "Z:\\Culture\\Videos"]

stash_raw_client.metadata_scan(videos_paths, {
    "scanGenerateClipPreviews": False,
    "scanGenerateCovers": True,
    "scanGenerateImagePreviews": False,
    "scanGeneratePhashes": True,
    "scanGeneratePreviews": False,
    "scanGenerateSprites": False,
    "scanGenerateThumbnails": False
})

'10'

In [ ]:
scenes_df = stash_client.find_scenes_by_studio(current_studio_ids)
print(len(scenes_df))

In [ ]:
import os
from libraries.file_renamer import create_filename

scene_renames_df = scenes_df.select([
    pl.col("stashapp_id"),
    pl.col("stashapp_primary_file_path").map_elements(lambda directory: os.path.dirname(directory), return_dtype=pl.Utf8).alias("directory"),
    pl.col("stashapp_primary_file_basename").alias("old_filename"),
    pl.struct(["*"]).map_elements(lambda row: create_filename(use_studio_code_tag, row), return_dtype=pl.Utf8).alias("new_filename"),
    pl.col("stashapp_primary_file_path").alias("old_path"),
]).with_columns([
    pl.col("new_filename").map_elements(lambda filename: len(filename) if filename else 0, return_dtype=pl.Int32).alias("new_filename_length"),
    pl.concat_str([
        pl.col("directory"),
        pl.lit(os.sep),
        pl.col("new_filename")
    ]).alias("new_path")
])

# Reorder the columns
scene_renames_df = scene_renames_df.select([
    pl.col("stashapp_id"),
    pl.col("directory"),
    pl.col("old_filename"),
    pl.col("new_filename"),
    pl.col("new_filename_length"),
    pl.col("old_path"),
    pl.col("new_path"),
])

scene_renames_df

In [ ]:
# Rename the files
import os
import polars as pl

scenes_success_rows = []
scenes_failed_rows = []

for row in scene_renames_df.iter_rows(named=True):
    old_path = row['old_path']
    new_path = row['new_path']
    failure_info = dict(row)  # Create a copy of the row data

    # Check for missing new path
    if new_path is None:
        failure_info['failure_reason'] = "Missing new path"
        failure_info['error_message'] = f"New path not found for {old_path}"
        scenes_failed_rows.append(failure_info)
        print(failure_info['error_message'])
        continue

    # Attempt to move the file if the old path is a file and the new path does not exist
    if os.path.isfile(old_path):
        if not os.path.exists(new_path):
            try:
                os.rename(old_path, new_path)

                # Check for and rename AI JSON sidecar file if it exists
                ai_json_path = f"{old_path}.AI.json"
                if os.path.isfile(ai_json_path):
                    new_ai_json_path = f"{new_path}.AI.json"
                    try:
                        os.rename(ai_json_path, new_ai_json_path)
                        print(f"Renamed AI JSON sidecar file:\n{ai_json_path}\n{new_ai_json_path}")
                    except Exception as e:
                        failure_info['failure_reason'] = "Sidecar rename failed"
                        failure_info['error_message'] = str(e)
                        scenes_failed_rows.append(failure_info)
                        continue
                print(f"Rename file:\n{old_path}\n{new_path}\n")
                scenes_success_rows.append(row)
            except Exception as e:
                failure_info['failure_reason'] = "Rename failed"
                failure_info['error_message'] = str(e)
                scenes_failed_rows.append(failure_info)
                print(f"Failed to rename:\n{old_path}\n{new_path}\n{e}")
        else:
            failure_info['failure_reason'] = "File exists"
            failure_info['error_message'] = f"A file already exists in the new path: {new_path}"
            scenes_failed_rows.append(failure_info)
            print(failure_info['error_message'])
    else:
        failure_info['failure_reason'] = "File not found"
        failure_info['error_message'] = f"File does not exist: {old_path}"
        scenes_failed_rows.append(failure_info)
        print(failure_info['error_message'])

scenes_success_df = pl.DataFrame(scenes_success_rows) if scenes_success_rows else pl.DataFrame(schema=scene_renames_df.schema)
scenes_failed_df = pl.DataFrame(scenes_failed_rows) if scenes_failed_rows else pl.DataFrame(schema={**scene_renames_df.schema, 'failure_reason': pl.Utf8, 'error_message': pl.Utf8})

In [ ]:
scenes_success_df

In [ ]:
scenes_failed_df

# Rename galleries

In [7]:
# Scan the galleries
gallery_paths = ["W:\\Culture\\Photos"]

stash_raw_client.metadata_scan(gallery_paths, {
    "scanGenerateClipPreviews": False,
    "scanGenerateCovers": True,
    "scanGenerateImagePreviews": False,
    "scanGeneratePhashes": True,
    "scanGeneratePreviews": False,
    "scanGenerateSprites": False,
    "scanGenerateThumbnails": False
})

'9'

In [4]:
galleries_df = stash_client.find_galleries_by_studio(current_studio_ids)
print(len(galleries_df))

217


In [5]:
import os
from libraries.file_renamer import create_filename

gallery_renames_df = galleries_df.select([
    pl.col("stashapp_id"),
    pl.col("stashapp_primary_file_path").map_elements(lambda directory: os.path.dirname(directory), return_dtype=pl.Utf8).alias("directory"),
    pl.col("stashapp_primary_file_basename").alias("old_filename"),
    pl.struct(["*"]).map_elements(lambda row: create_filename(use_studio_code_tag, row), return_dtype=pl.Utf8).alias("new_filename"),
    pl.col("stashapp_primary_file_path").alias("old_path"),
]).with_columns([
    pl.col("new_filename").map_elements(lambda filename: len(filename) if filename else 0, return_dtype=pl.Int32).alias("new_filename_length"),
    pl.concat_str([
        pl.col("directory"),
        pl.lit(os.sep),
        pl.col("new_filename")
    ]).alias("new_path")
])

# Reorder the columns
gallery_renames_df = gallery_renames_df.select([
    pl.col("stashapp_id"),
    pl.col("directory"),
    pl.col("old_filename"),
    pl.col("new_filename"),
    pl.col("new_filename_length"),
    pl.col("old_path"),
    pl.col("new_path"),
])

gallery_renames_df

stashapp_id,directory,old_filename,new_filename,new_filename_length,old_path,new_path
i64,str,str,str,i32,str,str
1249,"""W:\Culture\Photos\Vixen Networ…","""Blacked - 2017-06-29 - ive-nev…","""Vixen Media Group꞉ Blacked – 2…",202,"""W:\Culture\Photos\Vixen Networ…","""W:\Culture\Photos\Vixen Networ…"
1251,"""W:\Culture\Photos\Vixen Networ…","""Blacked - 2014-11-03 - interra…","""Vixen Media Group꞉ Blacked – 2…",157,"""W:\Culture\Photos\Vixen Networ…","""W:\Culture\Photos\Vixen Networ…"
1252,"""W:\Culture\Photos\Vixen Networ…","""Blacked - 2014-11-10 - two-tee…","""Vixen Media Group꞉ Blacked – 2…",154,"""W:\Culture\Photos\Vixen Networ…","""W:\Culture\Photos\Vixen Networ…"
1253,"""W:\Culture\Photos\Vixen Networ…","""Blacked - 2014-12-08 - perfect…","""Vixen Media Group꞉ Blacked – 2…",166,"""W:\Culture\Photos\Vixen Networ…","""W:\Culture\Photos\Vixen Networ…"
1254,"""W:\Culture\Photos\Vixen Networ…","""Blacked - 2015-03-22 - stunnin…","""Vixen Media Group꞉ Blacked – 2…",157,"""W:\Culture\Photos\Vixen Networ…","""W:\Culture\Photos\Vixen Networ…"
…,…,…,…,…,…,…
3734,"""W:\Culture\Photos\Vixen Networ…","""Blacked - 2024-05-03 - hotel-v…","""Vixen Media Group꞉ Blacked – 2…",157,"""W:\Culture\Photos\Vixen Networ…","""W:\Culture\Photos\Vixen Networ…"
3735,"""W:\Culture\Photos\Vixen Networ…","""Blacked - 2024-05-23 - low-rid…","""Vixen Media Group꞉ Blacked – 2…",214,"""W:\Culture\Photos\Vixen Networ…","""W:\Culture\Photos\Vixen Networ…"
3736,"""W:\Culture\Photos\Vixen Networ…","""Blacked - 2024-05-28 - break-t…","""Vixen Media Group꞉ Blacked – 2…",126,"""W:\Culture\Photos\Vixen Networ…","""W:\Culture\Photos\Vixen Networ…"


In [6]:
# Rename the files
import os
import polars as pl

galleries_success_rows = []
galleries_failed_rows = []

for row in gallery_renames_df.iter_rows(named=True):
    old_path = row['old_path']
    new_path = row['new_path']
    failure_info = dict(row)  # Create a copy of the row data

    # Check for missing new path
    if new_path is None:
        failure_info['failure_reason'] = "Missing new path"
        failure_info['error_message'] = f"New path not found for {old_path}"
        galleries_failed_rows.append(failure_info)
        print(failure_info['error_message'])
        continue

    # Attempt to move the file if the old path is a file and the new path does not exist
    if os.path.isfile(old_path):
        if not os.path.exists(new_path):
            try:
                os.rename(old_path, new_path)
                print(f"Rename file:\n{old_path}\n{new_path}\n")
                galleries_success_rows.append(row)
            except Exception as e:
                failure_info['failure_reason'] = "Rename failed"
                failure_info['error_message'] = str(e)
                galleries_failed_rows.append(failure_info)
                print(f"Failed to rename:\n{old_path}\n{new_path}\n{e}")
        else:
            failure_info['failure_reason'] = "File exists"
            failure_info['error_message'] = f"A file already exists in the new path: {new_path}"
            galleries_failed_rows.append(failure_info)
            print(failure_info['error_message'])
    else:
        failure_info['failure_reason'] = "File not found"
        failure_info['error_message'] = f"File does not exist: {old_path}"
        galleries_failed_rows.append(failure_info)
        print(failure_info['error_message'])

galleries_success_df = pl.DataFrame(galleries_success_rows) if galleries_success_rows else pl.DataFrame(schema=gallery_renames_df.schema)
galleries_failed_df = pl.DataFrame(galleries_failed_rows) if galleries_failed_rows else pl.DataFrame(schema={**gallery_renames_df.schema, 'failure_reason': pl.Utf8, 'error_message': pl.Utf8})

Rename file:
W:\Culture\Photos\Vixen Network - Blacked\Blacked - 2017-06-29 - ive-never-done-this-before - I've Never Done This Before - Kendra Sunderland, Ricky Johnson, Jason Brown, John Johnson, Isiah Maxwell & Nat Turnher [].zip
W:\Culture\Photos\Vixen Network - Blacked\Vixen Media Group꞉ Blacked – 2017-06-29 – I've Never Done This Before – Kendra Sunderland, Isiah Maxwell, Jason Brown, John Johnson, Nat Turnher, Ricky Johnson [018c0a41-83be-73b2-b6e6-1716546816cb].zip

Rename file:
W:\Culture\Photos\Vixen Network - Blacked\Blacked - 2014-11-03 - interracial-vacation-for-cheating-girlfriend - Interracial Vacation for Cheating Girlfriend - Remy LaCroix & Jason Brown [].zip
W:\Culture\Photos\Vixen Network - Blacked\Vixen Media Group꞉ Blacked – 2014-11-03 – Interracial Vacation For Cheating Girlfriend – Remy Lacroix, Jason Brown [018c0a65-482d-745d-b45c-2c4b48f04715].zip

Rename file:
W:\Culture\Photos\Vixen Network - Blacked\Blacked - 2014-11-10 - two-teen-girls-share-a-huge-bbc - Tw

In [ ]:
galleries_success_df

In [ ]:
galleries_failed_df